In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# 全局设置
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.linewidth'] = 0.6  # 调整坐标轴线宽
plt.rcParams['figure.figsize'] = (3, 4)  # 调整图表尺寸


# 定义函数

In [ ]:

def plot_combined_figure(betaout, signal_names, area):
    num_signals = len(signal_names)
    fig, (ax1, ax2) = plt.subplots(1, 2, dpi=1200, gridspec_kw={'height_ratios': [1.5]},figsize=(4.5, 2))
    offsets = np.linspace(-0.1, 0.1, num_signals)  # 调整偏移

    # 绘制信号的缩放系数及置信区间
    for i, signal in enumerate(signal_names):
        beta_low = f'beta{i+1}_low'
        beta_hat = f'beta{i+1}_hat'
        beta_up = f'beta{i+1}_up'
        color = signal_colors[signal]

        ax1.errorbar(
            betaout['#EOF'] + offsets[i], betaout[beta_hat],
            yerr=[betaout[beta_hat] - betaout[beta_low], betaout[beta_up] - betaout[beta_hat]],
            fmt='o', color=color, label=signal, capsize=2, linewidth=0.8,markersize=2
        )
        for j in range(len(betaout)):
            ax1.plot(
                [betaout['#EOF'][j] + offsets[i], betaout['#EOF'][j] + offsets[i]],
                [betaout[beta_low][j], betaout[beta_up][j]], color=color, linewidth=0.6
            )

    ax1.axhline(0, color='black', linewidth=0.6)
    ax1.axhline(1, linestyle='--', color='gray', linewidth=0.6)
    ax1.axhline(-1, linestyle='--', color='gray', linewidth=0.6)
    ax1.set_ylabel("Scaling factors", fontsize=8)
    ax1.set_xlabel("Number of EOF patterns", fontsize=8)
    ax1.set_xticks(np.arange(4, 8, 1))  # 以 1 为间隔设置 x 轴刻度（2,3,4,5,6,7）
    ax1.set_yticks(np.arange(-8, 8, 2))
    ax1.tick_params(axis='both', which='both',direction='in', bottom=True, labelbottom=True, labelsize=7)
    ax1.legend(fontsize=6, loc='best')
    ax1.set_title("Scaling factors", fontsize=9)
    ax1.text(0.97, 1.08, area, fontsize=6,transform=ax1.transAxes,va='top', ha='right')
    ax1.text(-.04, 1.15, 'a', transform=ax1.transAxes, fontsize=10, fontweight='bold', va='top', ha='right')
    ax1.set_ylim([-8,8])

    # 绘制残差一致性测试
    yrange = [
        (1 / betaout[['RCstat', 'RClow', 'RCup']]).min().min(),
        (1 / betaout[['RCstat', 'RClow', 'RCup']]).max().max()
    ]

    ax2.set_yscale('log')
    ax2.plot(betaout['#EOF'], 1 / betaout['RCstat'], 'o', color="black", label="RCstat", markersize=3)
    ax2.plot(betaout['#EOF'], 1 / betaout['RClow'], 'b--', label="RClow", linewidth=0.6)
    ax2.plot(betaout['#EOF'], 1 / betaout['RCup'], 'b--', label="RCup", linewidth=0.6)
    ax2.axhline(1, color='gray', linewidth=0.5, alpha=0.5)

    ax2.set_ylim(yrange)
    ax2.set_xlabel("Number of EOF patterns", fontsize=8)
    ax2.set_ylabel("Model/obs variance", fontsize=8)
    ax2.tick_params(axis='both', direction='in', length=3, width=0.6, labelsize=7)
    #ax2.minorticks_on()
    ax2.tick_params(axis='y', which='minor', direction='in', length=2, width=0.6, right=False, left=True)  # 让副轴刻度线向里
    ax2.set_title("Residual consistency", fontsize=9)
    ax2.text(-.04, 1.15, 'b', transform=ax2.transAxes, fontsize=10, fontweight='bold', va='top', ha='right')
    ax2.set_xticks(np.arange(4, 8, 1))  # 以 1 为间隔设置 x 轴刻度（2,3,4,5,6,7）

    plt.tight_layout()

# 绘制图像

In [ ]:
# 调整配色方案
signal_colors = {
    'ALL': '#E31A1C',  # 活力红
    'GHG': '#4DAF4A',   # 浅绿色
    'AER': '#DA70D6',  # 鲜艳紫
    'NAT': '#FF7F00'  # 亮黄色
}


# 文件路径和文件名列表
base_dir = "/home/dongyl/UPWARD_SHIFT_OF_JET_STREAM_DATAFILES/fingerprint/1980-2019/"
data_files = [
    "data_S_GHG_aer_nat", "data_N_GHG_aer_nat", "data_S_GHG_aer", "data_N_GHG_aer",
    "data_S_all", "data_N_all", "data_S_GHG", "data_N_GHG",
    "data_S_aer", "data_N_aer", "data_S_nat", "data_N_nat"
]

# 信号名称列表
signal_names_dict = {
    "data_S_GHG_aer_nat": ['GHG', 'AER', 'NAT'],
    "data_N_GHG_aer_nat": ['GHG', 'AER', 'NAT'],
    "data_S_GHG_aer": ['GHG', 'AER'],
    "data_N_GHG_aer": ['GHG', 'AER'],
    "data_S_all": ['ALL'],
    "data_N_all": ['ALL'],
    "data_S_GHG": ['GHG'],
    "data_N_GHG": ['GHG'],
    "data_S_aer": ['AER'],
    "data_N_aer": ['AER'],
    "data_S_nat": ['NAT'],
    "data_N_nat": ['NAT']
}
#data_files = ["obs_GHG_aer_nat_ens3run_20_40_N_1958-2017_sf_RC"]
data_files = ["obs_GHG_aer_nat_ens3run_20_40_N_1980-2019_sf_RC"]
# 循环读取文件并绘图
for data_file in data_files:
    file_path = os.path.join(base_dir, f"{data_file}.csv")

    # 读取 CSV 文件
    df = pd.read_csv(file_path)

    # 获取信号名称
    #signal_names = signal_names_dict.get(data_file, [])
    signal_names = ['GHG', 'AER', 'NAT']
    # 根据文件名判断是北半球还是南半球
    if "_N_" in data_file:
        hemisphere_label = "20-40°N"
    elif "_S_" in data_file:
        hemisphere_label = "20-40°S"
    else:
        hemisphere_label = ""

    # 调用绘图函数
    plot_combined_figure(df, signal_names, hemisphere_label)

    # 修改输出文件名，去掉前缀 "data_"
    output_filename = f"20-40_{data_file.replace('data_', '')}.svg"
    output_path = os.path.join(base_dir, output_filename)

    # 保存绘制的图像
    #plt.savefig(output_path, dpi=500)
    plt.savefig(output_path, format='svg',dpi=1200)
    print(f"Saved figure to {output_path}")
    plt.show()

    # 清理图像缓存
    plt.clf()